In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **QUESTION 3**
Seq to seq model implementation
Train the following models separately as encoder-decoder frameworks for machine translation:
1. RNN-Based Sequence-to-Sequence Model
2. Bi-directional RNN Model
3. LSTM-Based Sequence-to-Sequence Model
4. Transformer-Based Model (using multi-head attention)
To ensure a fair comparison between the RNN, BiRNN, LSTM, BiLSTM, and Transformer
models, all models must use the same core training configurations. Train each model for at least
50 epochs. Vocabulary size and padding must be consistent across all models. You can experiment
with different hyperparameters (number of layers, drop out rate, optimizer) for better results and
report all hyperparameters as a table.
Evaluation and Comparison
1. Translate a fixed set of test English sentences to Urdu using all five models.
2. Evaluate using BLEU Score
3. Record and compare:
o Final scores (BLEU)
o Inference examples

In [12]:
!pip install -q nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
nltk.download('punkt')
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, SimpleRNN, LSTM, Dense, Bidirectional, GRU, MultiHeadAttention, LayerNormalization, Dropout, Add
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
# Load the files
with open('/content/drive/MyDrive/Colab Notebooks/english-corpus.txt', 'r', encoding='utf-8') as f:
    english_lines = f.read().splitlines()
with open('/content/drive/MyDrive/Colab Notebooks/urdu-corpus.txt', 'r', encoding='utf-8') as f:
    urdu_lines = f.read().splitlines()

# Create DataFrame
df = pd.DataFrame({'english': english_lines, 'urdu': urdu_lines}).dropna()

# Add <sos> and <eos> tokens to Urdu (target)
df['urdu'] = df['urdu'].apply(lambda x: '<sos> ' + x + ' <eos>')

# Tokenization
eng_tokenizer = Tokenizer(oov_token='<OOV>', filters='')
urdu_tokenizer = Tokenizer(oov_token='<OOV>', filters='')

eng_tokenizer.fit_on_texts(df['english'])
urdu_tokenizer.fit_on_texts(df['urdu'])

# Convert to sequences
input_seq = eng_tokenizer.texts_to_sequences(df['english'])
target_seq = urdu_tokenizer.texts_to_sequences(df['urdu'])

# Padding
max_input_len = max(len(seq) for seq in input_seq)
max_target_len = max(len(seq) for seq in target_seq)

encoder_input = pad_sequences(input_seq, maxlen=max_input_len, padding='post')
decoder_input = pad_sequences(target_seq, maxlen=max_target_len, padding='post')

# Prepare decoder target (right-shifted decoder input)
decoder_target = np.zeros_like(decoder_input)
decoder_target[:, :-1] = decoder_input[:, 1:]


In [14]:
VOCAB_SIZE_EN = len(eng_tokenizer.word_index) + 1
VOCAB_SIZE_UR = len(urdu_tokenizer.word_index) + 1
EMBEDDING_DIM = 256
UNITS = 512
EPOCHS = 50
BATCH_SIZE = 64

In [15]:
def build_seq2seq_model(cell_type='rnn', bidirectional=False):
    # Encoder
    encoder_inputs = Input(shape=(max_input_len,))
    encoder_embedding = Embedding(VOCAB_SIZE_EN, EMBEDDING_DIM)(encoder_inputs)

    if cell_type == 'lstm':
        if bidirectional:
            encoder_outputs, forward_h, forward_c, backward_h, backward_c = Bidirectional(
                LSTM(UNITS, return_state=True)
            )(encoder_embedding)
            state_h = Concatenate()([forward_h, backward_h])
            state_c = Concatenate()([forward_c, backward_c])
        else:
            encoder_outputs, state_h, state_c = LSTM(UNITS, return_state=True)(encoder_embedding)
    else:
        if bidirectional:
            encoder_outputs, forward_h, backward_h = Bidirectional(
                SimpleRNN(UNITS, return_state=True)
            )(encoder_embedding)
            state_h = Concatenate()([forward_h, backward_h])
            state_c = None
        else:
            encoder_outputs, state_h = SimpleRNN(UNITS, return_state=True)(encoder_embedding)
            state_c = None

    # Decoder
    decoder_inputs = Input(shape=(max_target_len,))
    decoder_embedding = Embedding(VOCAB_SIZE_UR, EMBEDDING_DIM)(decoder_inputs)

    if cell_type == 'lstm':
        if bidirectional:
            decoder_lstm = LSTM(UNITS * 2, return_sequences=True)
            decoder_outputs = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])
        else:
            decoder_lstm = LSTM(UNITS, return_sequences=True)
            decoder_outputs = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])
    else:
        if bidirectional:
            decoder_rnn = SimpleRNN(UNITS * 2, return_sequences=True)
            decoder_outputs = decoder_rnn(decoder_embedding, initial_state=[state_h])
        else:
            decoder_rnn = SimpleRNN(UNITS, return_sequences=True)
            decoder_outputs = decoder_rnn(decoder_embedding, initial_state=[state_h])

    decoder_dense = Dense(VOCAB_SIZE_UR, activation='softmax')
    output = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], output)
    return model


In [16]:
def build_transformer_model():
    inputs = Input(shape=(max_input_len,))
    x = Embedding(VOCAB_SIZE_EN, EMBEDDING_DIM)(inputs)
    x = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
    x = LayerNormalization()(x)
    x = Dropout(0.1)(x)
    x = Dense(UNITS, activation='relu')(x)
    x = Dense(VOCAB_SIZE_UR, activation='softmax')(x)
    model = Model(inputs, x)
    return model


In [17]:
def train_model(model, model_name):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(
        [encoder_input, decoder_input], decoder_target,
        batch_size=BATCH_SIZE, epochs=EPOCHS,
        validation_split=0.2
    )
    model.save(f"{model_name}_model.h5")

from nltk.translate.bleu_score import sentence_bleu
import numpy as np

def evaluate_bleu(model, sample_indices):
    bleu_scores = []

    for idx in sample_indices:
        input_seq = encoder_input[idx:idx+1]
        true_output = df['urdu'].iloc[idx].replace('<sos>', '').replace('<eos>', '')
        prediction = model.predict([input_seq, decoder_input[idx:idx+1]])
        predicted_seq = np.argmax(prediction[0], axis=1)
        predicted_words = [urdu_tokenizer.index_word.get(i, '') for i in predicted_seq if i != 0]
        pred_text = ' '.join(predicted_words).replace('<sos>', '').replace('<eos>', '')

        bleu = sentence_bleu([true_output.split()], pred_text.split())
        bleu_scores.append(bleu)

        print(f"\n🔹Input: {df['english'].iloc[idx]}")
        print(f"🔹Target: {true_output}")
        print(f"🔹Predicted: {pred_text}")
        print(f"🔹BLEU Score: {bleu:.4f}")

    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    print(f"\n✨ Average BLEU Score: {avg_bleu:.4f}")




In [17]:
# RNN
rnn_model = build_seq2seq_model('rnn')
train_model(rnn_model, 'rnn')
evaluate_bleu(rnn_model, [10, 20, 30])


Epoch 1/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 19s 41ms/step - accuracy: 0.7380 - loss: 2.0880 - val_accuracy: 0.7892 - val_loss: 1.2861
Epoch 2/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - accuracy: 0.7926 - loss: 1.2209 - val_accuracy: 0.8032 - val_loss: 1.1545
Epoch 3/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - accuracy: 0.8075 - loss: 1.0699 - val_accuracy: 0.8073 - val_loss: 1.1015
Epoch 4/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - accuracy: 0.8152 - loss: 0.9774 - val_accuracy: 0.8138 - val_loss: 1.0770
Epoch 5/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - accuracy: 0.8214 - loss: 0.9153 - val_accuracy: 0.8154 - val_loss: 1.0729
Epoch 6/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - accuracy: 0.8267 - loss: 0.8660 - val_accuracy: 0.8167 - val_loss: 1.0734
Epoch 7/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - accuracy: 0.8316 - loss: 0.8255 - val_accuracy: 0.8170 - val_loss: 1.0774
Epoch 8/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - accuracy: 0.8373 - loss: 0.7917 - val_

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 561ms/step

🔹Input: zain was hesitant
🔹Target:  زین ہچکچا رہا تھا 
🔹Predicted: میں نے رہا تھا 
🔹BLEU Score: 0.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step

🔹Input: did zain give you that
🔹Target:  زین نے تمہیں وہ دیا 
🔹Predicted: میں نے مریم معاف دیا 
🔹BLEU Score: 0.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

🔹Input: i come from china
🔹Target:  میں چین سے آیا ہوں۔ 
🔹Predicted: میں نے سے ہوں ہوں۔ 
🔹BLEU Score: 0.0000

✨ Average BLEU Score: 0.0000


/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

**Bi-RNN**

In [18]:
from tensorflow.keras.layers import Concatenate

def build_bi_rnn_model():
    # Encoder
    encoder_inputs = Input(shape=(max_input_len,))
    encoder_embedding = Embedding(VOCAB_SIZE_EN, EMBEDDING_DIM)(encoder_inputs)

    # Bidirectional RNN Encoder
    encoder_rnn = Bidirectional(SimpleRNN(UNITS, return_state=True))
    encoder_outputs, forward_h, backward_h = encoder_rnn(encoder_embedding)
    state_h = Concatenate()([forward_h, backward_h])

    # Decoder
    decoder_inputs = Input(shape=(max_target_len,))
    decoder_embedding = Embedding(VOCAB_SIZE_UR, EMBEDDING_DIM)(decoder_inputs)

    decoder_rnn = SimpleRNN(UNITS * 2, return_sequences=True)
    decoder_outputs = decoder_rnn(decoder_embedding, initial_state=[state_h])

    decoder_dense = Dense(VOCAB_SIZE_UR, activation='softmax')
    output = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], output)
    return model


In [19]:
# Build and train the Bi-RNN model
bi_rnn_model = build_bi_rnn_model()
train_model(bi_rnn_model, 'bi_rnn')

# Evaluate the Bi-RNN model with BLEU scores
evaluate_bleu(bi_rnn_model, [10, 20, 30])


Epoch 1/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 26s 62ms/step - accuracy: 0.7404 - loss: 1.9111 - val_accuracy: 0.8020 - val_loss: 1.2275
Epoch 2/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 34s 49ms/step - accuracy: 0.8120 - loss: 1.1265 - val_accuracy: 0.8360 - val_loss: 0.9898
Epoch 3/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 20s 49ms/step - accuracy: 0.8444 - loss: 0.8610 - val_accuracy: 0.8506 - val_loss: 0.8893
Epoch 4/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 20s 49ms/step - accuracy: 0.8622 - loss: 0.6883 - val_accuracy: 0.8575 - val_loss: 0.8424
Epoch 5/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step - accuracy: 0.8766 - loss: 0.5594 - val_accuracy: 0.8634 - val_loss: 0.8200
Epoch 6/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 21s 49ms/step - accuracy: 0.8925 - loss: 0.4506 - val_accuracy: 0.8680 - val_loss: 0.8124
Epoch 7/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 21s 49ms/step - accuracy: 0.9066 - loss: 0.3690 - val_accuracy: 0.8709 - val_loss: 0.8216
Epoch 8/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 16s 51ms/step - accuracy: 0.9208 - loss: 0.2995 - 

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 988ms/step

🔹Input: zain was hesitant
🔹Target:  زین ہچکچا رہا تھا 
🔹Predicted: زین ہچکچا رہا تھا 
🔹BLEU Score: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step

🔹Input: did zain give you that
🔹Target:  زین نے تمہیں وہ دیا 
🔹Predicted: زین نے تمہیں وہ دیا 
🔹BLEU Score: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step

🔹Input: i come from china
🔹Target:  میں چین سے آیا ہوں۔ 
🔹Predicted: میں چین سے آیا ہوں۔ 
🔹BLEU Score: 1.0000

✨ Average BLEU Score: 1.0000


**LSTM**

In [20]:
def build_lstm_model():
    # Encoder
    encoder_inputs = Input(shape=(max_input_len,))
    encoder_embedding = Embedding(VOCAB_SIZE_EN, EMBEDDING_DIM)(encoder_inputs)

    # Encoder LSTM
    encoder_lstm = LSTM(UNITS, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)

    # Decoder
    decoder_inputs = Input(shape=(max_target_len,))
    decoder_embedding = Embedding(VOCAB_SIZE_UR, EMBEDDING_DIM)(decoder_inputs)

    decoder_lstm = LSTM(UNITS, return_sequences=True)
    decoder_outputs = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])

    decoder_dense = Dense(VOCAB_SIZE_UR, activation='softmax')
    output = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], output)
    return model


In [21]:
# Build and train the LSTM model
lstm_model = build_lstm_model()
train_model(lstm_model, 'lstm')

# Evaluate the LSTM model with BLEU scores
evaluate_bleu(lstm_model, [10, 20, 30])


Epoch 1/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 21s 49ms/step - accuracy: 0.7342 - loss: 2.1701 - val_accuracy: 0.7863 - val_loss: 1.3118
Epoch 2/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step - accuracy: 0.7934 - loss: 1.2405 - val_accuracy: 0.8097 - val_loss: 1.1465
Epoch 3/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 16s 52ms/step - accuracy: 0.8157 - loss: 1.0705 - val_accuracy: 0.8277 - val_loss: 1.0324
Epoch 4/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 19s 48ms/step - accuracy: 0.8347 - loss: 0.9302 - val_accuracy: 0.8419 - val_loss: 0.9422
Epoch 5/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 20s 47ms/step - accuracy: 0.8499 - loss: 0.8147 - val_accuracy: 0.8527 - val_loss: 0.8740
Epoch 6/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step - accuracy: 0.8630 - loss: 0.7115 - val_accuracy: 0.8609 - val_loss: 0.8204
Epoch 7/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 16s 52ms/step - accuracy: 0.8749 - loss: 0.6216 - val_accuracy: 0.8665 - val_loss: 0.7758
Epoch 8/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 16s 51ms/step - accuracy: 0.8857 - loss: 0.5394 - 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step

🔹Input: zain was hesitant
🔹Target:  زین ہچکچا رہا تھا 
🔹Predicted: زین ہچکچا رہا تھا 
🔹BLEU Score: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

🔹Input: did zain give you that
🔹Target:  زین نے تمہیں وہ دیا 
🔹Predicted: زین نے تمہیں وہ دیا 
🔹BLEU Score: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step

🔹Input: i come from china
🔹Target:  میں چین سے آیا ہوں۔ 
🔹Predicted: میں چین سے آیا ہوں۔ 
🔹BLEU Score: 1.0000

✨ Average BLEU Score: 1.0000


**TRANSFORMER**

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class PositionalEncoding(Layer):
    def __init__(self, position, d_model):
        super().__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_config(self):
        return {"position": self.pos_encoding.shape[0], "d_model": self.pos_encoding.shape[1]}

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            np.arange(position)[:, np.newaxis],
            np.arange(d_model)[np.newaxis, :],
            d_model)
        # apply sin to even indices and cos to odd indices
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        pos_encoding = angle_rads[np.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)

    def get_angles(self, pos, i, d_model):
        return pos / np.power(10000, (2 * (i // 2)) / np.float32(d_model))

    def call(self, x):
        return x + self.pos_encoding[:, :tf.shape(x)[1], :]


In [2]:
from tensorflow.keras.layers import (
    Input, Dense, Dropout, Embedding, LayerNormalization,
    MultiHeadAttention, Add
)

def transformer_encoder_layer(embed_dim, num_heads, ff_dim, rate=0.1):
    inputs = Input(shape=(None, embed_dim))
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(inputs, inputs)
    attn_output = Dropout(rate)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(Add()([inputs, attn_output]))

    ffn = Dense(ff_dim, activation='relu')(out1)
    ffn = Dense(embed_dim)(ffn)
    ffn_output = Dropout(rate)(ffn)
    out2 = LayerNormalization(epsilon=1e-6)(Add()([out1, ffn_output]))

    return tf.keras.Model(inputs=inputs, outputs=out2)

def transformer_decoder_layer(embed_dim, num_heads, ff_dim, rate=0.1):
    inputs = Input(shape=(None, embed_dim))
    enc_output = Input(shape=(None, embed_dim))

    attn1 = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(inputs, inputs)
    attn1 = Dropout(rate)(attn1)
    out1 = LayerNormalization(epsilon=1e-6)(Add()([inputs, attn1]))

    attn2 = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(out1, enc_output)
    attn2 = Dropout(rate)(attn2)
    out2 = LayerNormalization(epsilon=1e-6)(Add()([out1, attn2]))

    ffn = Dense(ff_dim, activation='relu')(out2)
    ffn = Dense(embed_dim)(ffn)
    ffn_output = Dropout(rate)(ffn)
    out3 = LayerNormalization(epsilon=1e-6)(Add()([out2, ffn_output]))

    return tf.keras.Model(inputs=[inputs, enc_output], outputs=out3)


In [3]:
def build_transformer_model():
    embed_dim = EMBEDDING_DIM
    ff_dim = 512
    num_heads = 4
    dropout_rate = 0.1

    # Encoder
    encoder_inputs = Input(shape=(max_input_len,))
    x = Embedding(VOCAB_SIZE_EN, embed_dim)(encoder_inputs)
    x = PositionalEncoding(max_input_len, embed_dim)(x)
    encoder_layer = transformer_encoder_layer(embed_dim, num_heads, ff_dim, dropout_rate)
    encoder_output = encoder_layer(x)

    # Decoder
    decoder_inputs = Input(shape=(max_target_len,))
    y = Embedding(VOCAB_SIZE_UR, embed_dim)(decoder_inputs)
    y = PositionalEncoding(max_target_len, embed_dim)(y)
    decoder_layer = transformer_decoder_layer(embed_dim, num_heads, ff_dim, dropout_rate)
    decoder_output = decoder_layer([y, encoder_output])

    final_output = Dense(VOCAB_SIZE_UR, activation='softmax')(decoder_output)

    model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=final_output)
    return model


In [ ]:
# Build and train the Transformer model
transformer_model = build_transformer_model()
train_model(transformer_model, 'transformer')

# Evaluate the Transformer model with BLEU scores
evaluate_bleu(transformer_model, [10, 20, 30])

Epoch 1/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 47s 86ms/step - accuracy: 0.7360 - loss: 2.2419 - val_accuracy: 0.8544 - val_loss: 0.9484
Epoch 2/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 12s 37ms/step - accuracy: 0.8967 - loss: 0.6842 - val_accuracy: 0.9649 - val_loss: 0.2598
Epoch 3/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - accuracy: 0.9719 - loss: 0.2003 - val_accuracy: 0.9816 - val_loss: 0.1492
Epoch 4/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 12s 37ms/step - accuracy: 0.9869 - loss: 0.0923 - val_accuracy: 0.9868 - val_loss: 0.1189
Epoch 5/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 20s 37ms/step - accuracy: 0.9921 - loss: 0.0527 - val_accuracy: 0.9889 - val_loss: 0.1069
Epoch 6/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - accuracy: 0.9954 - loss: 0.0288 - val_accuracy: 0.9903 - val_loss: 0.1034
Epoch 7/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 21s 37ms/step - accuracy: 0.9981 - loss: 0.0145 - val_accuracy: 0.9904 - val_loss: 0.1059
Epoch 8/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 21s 38ms/step - accuracy: 0.9993 - loss: 0.0079 - 

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

🔹Input: zain was hesitant
🔹Target:  زین ہچکچا رہا تھا 
🔹Predicted: زین ہچکچا رہا تھا 
🔹BLEU Score: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step

🔹Input: did zain give you that
🔹Target:  زین نے تمہیں وہ دیا 
🔹Predicted: زین نے تمہیں وہ دیا 
🔹BLEU Score: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step

🔹Input: i come from china
🔹Target:  میں چین سے آیا ہوں۔ 
🔹Predicted: میں چین سے آیا ہوں۔ 
🔹BLEU Score: 1.0000
